In [7]:
import lxml.html
from pathlib import Path
import pickle
import re
import time
from tqdm import tqdm

from projectmir.xmldocument import XMLDocument
from projectmir.functions import find_identifier_definition

In [11]:
p_path = './data/test_latexml/'
p = Path(p_path)
x = list(p.glob('*.html'))
documents_IDs = [x_.name[:-5] for x_ in x]

doc = [[]]*len(documents_IDs)
for i, documents_ID in tqdm(enumerate(documents_IDs)):
    documents_path = p_path + documents_ID + '.html'
    doc_ = find_identifier_definition(documents_path)
    doc[i] = doc_

save_filename = 'doc.pkl' + time.strftime('%d%b%Y_%H%M%S')
with open(save_filename, 'wb') as f:
    pickle.dump(doc, f)


0it [00:00, ?it/s]elapsed time : 0.0000110 seconds ---
process document
elapsed time : 0.0028889 seconds ---
Number of math components is 50
mover====================
['H', '^']
+++++++++++++++++++++++++
mover====================
['H', '^']
+++++++++++++++++++++++++
mover====================
['H', '^']
+++++++++++++++++++++++++
mover====================
['H', '^']
+++++++++++++++++++++++++
mover====================
['H', '^']
+++++++++++++++++++++++++
mover====================
['H', '^']
+++++++++++++++++++++++++
mover====================
['H', '^']
+++++++++++++++++++++++++
elapsed time : 0.0470998 seconds ---
elapsed time : 0.0894191 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-b7a25e8ae1b84f0c.props -preload tokenize,ssplit,pos
elapse

In [9]:
load_filename = 'doc.pkl'
with open(load_filename, 'rb') as f:
    doc = pickle.load(f)

In [10]:
print(doc[0].path)
print(doc[0].identifiers)

./data/test_latexml/test_sample.html
['w_i', 'T_i', 'Q', 'T', 'w', 'V', 'ρ', 'C', 'U_int', 'H', 'd', 't', 'T_ref', 'Δ', 'N_F']


In [13]:
tree = lxml.html.parse(doc[0].path)

In [14]:
html = tree.getroot()

In [6]:
x_list = html.cssselect('math')

NameError: name 'html' is not defined

In [5]:
for i, x in enumerate(x_list):
    print(i)
    print(lxml.html.tostring(x.drop_tree))


NameError: name 'x_list' is not defined

In [31]:
[x.text_content() for x in x_list[13].cssselect('mover')[0].iterchildren()]

['U', '^']

In [32]:
x_list[13].text_content()

<bound method HtmlMixin.text_content of <Element math at 0x12ba98040>>